In [1]:
import pandas as pd
import glob
import os
    
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from Bio import SeqIO

import xmltodict
from flatten_json import flatten
import json

import requests

In [15]:
def get_superfamily_cds(superfamily_pssm):
    # search
    r = requests.get(f'https://www.ncbi.nlm.nih.gov/Structure/cdd/cddsrv.cgi?uid={superfamily_pssm}')
    # get text
    query_dict = json.loads((r.text.split(' CDD=')[1].split(';')[0]))
    # get CDs in the superfamily
    curated_cds, uncurated_cds = [], []
    if 'curatedCDs' in query_dict.keys():
        curated_cds = [cd[0] for cd in query_dict['curatedCDs']]
    if 'unCuratedCDs' in query_dict.keys():
        uncurated_cds = [cd[0] for cd in query_dict['unCuratedCDs']]
        
    return curated_cds + uncurated_cds

## SparcleLabel ribosomal protein annotation process

### Steps
1. Get cddid_tbl.gz
```
wget https://ftp.ncbi.nih.gov/pub/mmdb/cdd/cddid.tbl.gz
```
on May 26, 2022
2. Get cdd
```
wget cdd.tar.gz
tar -xzf cdd.tar.gz
mkdir smp
mv *.smp smp
```

In [3]:
cddid_tbl = pd.read_csv('/data/mhoffert/db/cdd/cddid.tbl', sep='\t', header=None)
cddid_tbl.columns = '''PSSM-Id
CD accession
CD short name
CD description
PSSM-Length'''.split('\n')

In [4]:
psms = glob.glob('/data/mhoffert/db/cdd/smp/*')

In [7]:
ribosomal_proteins = cddid_tbl[cddid_tbl['CD description'].apply(str.lower).str.contains('ribosomal protein')]

In [8]:
# # count = 0
# potential_rps = cddid_tbl[cddid_tbl['CD description'].apply(str.lower).str.contains('ribosomal protein')].iloc[count:, :]
# # validated_rps = []
# for index, row in potential_rps.iterrows():
    
#     display(f'{count} out of {len(potential_rps)}')
#     display(row['CD description'])
#     display(row['CD short name'])
#     is_ribosomal = ''
#     while not is_ribosomal in ['y', 'n']:
#         is_ribosomal = input("Is this a ribosomal protein? y/n:")
#         display('Try again')
    
#     if is_ribosomal == 'y':
#         validated_rps.append(row)
#     clear_output(wait=True)
#     count += 1

In [15]:
# ribosomal_proteins = pd.concat(validated_rps, axis=1).T

In [16]:
ribosomal_proteins.to_csv('/data/mhoffert/realmichaelhoffert/ribosomal_proteins/data/validated_ribosomal_proteins_cdd.tsv', sep='\t')

In [150]:
ribosomal_proteins = pd.read_csv('/data/mhoffert/realmichaelhoffert/ribosomal_proteins/data/validated_ribosomal_proteins_cdd.tsv', sep='\t', index_col=0)

In [16]:
ribosomal_protein_accs = list(ribosomal_proteins['CD accession'].unique())

# get the superfamilies (ids starting with "cl")
superfamily_list = [protein for protein in ribosomal_protein_accs if not f'/data/mhoffert/db/cdd/smp/{protein}.smp' in psms]
superfamily_pssm_ids = ribosomal_proteins[ribosomal_proteins['CD accession'].isin(superfamily_list)]['PSSM-Id'].unique()

# add CDs of superfamilies
for i, superfamily in enumerate(superfamily_pssm_ids):
    display(i)
    clear_output(wait=True)
    ribosomal_protein_accs += get_superfamily_cds(superfamily)

# unique    
ribosomal_protein_accs = list(set(ribosomal_protein_accs))

smp_list = [f'/data/mhoffert/db/cdd/smp/{protein}.smp' for protein in ribosomal_protein_accs if f'/data/mhoffert/db/cdd/smp/{protein}.smp' in psms]
print(f'Num found: {len(smp_list) + len(superfamily_pssm_ids)}, num searched: {len(ribosomal_protein_accs)}')

Num found: 925, num searched: 925


In [18]:
len(smp_list)

834

In [17]:
with open('/data/mhoffert/db/cdd/ribosomal_proteins.pn', 'w') as pn_file:
    pn_file.write('\n'.join(smp_list))

```
~/tools/SparcleLabel/SparcleLabel-x64-linux/ncbi-blast-2.13.0+/bin/makeprofiledb -title ribosomal_proteins -in ribosomal_proteins.pn -out ribosomal_proteins -threshold 9.82 -scale 100.0 -dbtype rps -index true
```

## PGAP HMM ribosomal protein process

### Steps
1. Get PGAP hmms and unpack
```
wget https://ftp.ncbi.nih.gov/hmm/current/hmm_PGAP.HMM.tgz
tar -xzvf hmm_PGAP.HMM.tgz
```
2. Read tsv file
```
https://ftp.ncbi.nih.gov/hmm/current/hmm_PGAP.tsv
```

In [19]:
pgap_hmm = pd.read_csv('/data/mhoffert/db/pgap_ribosomal/hmm_PGAP.tsv', sep='\t')

In [20]:
pgap_hmm.family_type.value_counts()

equivalog                  10928
domain                      8161
subfamily                   3363
PfamEq                      2260
PfamAutoEq                  1217
exception                    918
hypoth_equivalog             456
equivalog_domain             204
subfamily_domain             198
repeat                       117
hypoth_equivalog_domain       39
paralog                       28
superfamily                   27
paralog_domain                15
signature                      5
Name: family_type, dtype: int64

In [21]:
hmms = glob.glob('/data/mhoffert/db/pgap_ribosomal/hmm_PGAP/*.HMM')

In [22]:
ribosomal_hmms = pgap_hmm[pgap_hmm.product_name.apply(str.lower).str.contains('ribosomal protein')]

In [95]:
# count = 0
# potential_hmms = pgap_hmm[pgap_hmm.product_name.apply(str.lower).str.contains('ribosomal protein')]
# validated_hmms = []
# for index, row in potential_hmms.iterrows():
    
#     display(f'{count} out of {len(potential_hmms)}')
#     display(row['label'])
#     display(row['name_orig'])
#     is_ribosomal = ''
#     while not is_ribosomal in ['y', 'n']:
#         is_ribosomal = input("Is this a ribosomal protein? y/n:")
#         display('Try again')
    
#     if is_ribosomal == 'y':
#         validated_hmms.append(row)
#     clear_output(wait=True)
#     count += 1

# ribosomal_hmms = pd.concat(validated_hmms, axis=1).T

# ribosomal_hmms.to_csv('/data/mhoffert/realmichaelhoffert/ribosomal_proteins/data/validated_ribosomal_proteins_hmms.tsv', sep='\t')

# ribosomal_hmms = pd.read_csv('/data/mhoffert/realmichaelhoffert/ribosomal_proteins/data/validated_ribosomal_proteins_hmms.tsv', sep='\t')

In [23]:
ribosomal_hmms_accs = ribosomal_hmms['#ncbi_accession'].unique()

In [24]:
hmm_list = [f'/data/mhoffert/db/pgap_ribosomal/hmm_PGAP/{acc}.HMM' for acc in ribosomal_hmms_accs if f'/data/mhoffert/db/pgap_ribosomal/hmm_PGAP/{acc}.HMM' in hmms]

In [25]:
not_matching_hmm_list = [acc for acc in ribosomal_hmms_accs if not f'/data/mhoffert/db/pgap_ribosomal/hmm_PGAP/{acc}.HMM' in hmms]

In [26]:
len(hmm_list), len(ribosomal_hmms_accs), len(not_matching_hmm_list)

(245, 336, 91)

In [28]:
for i, hmm in enumerate(hmm_list):
    display(i)
    clear_output(wait=True)
    os.system(f'cat {hmm} >> /data/mhoffert/db/pgap_ribosomal/ribosomal_hmm')

244

Press HMMs according to these instructions: http://eddylab.org/software/hmmer3/3.1b2/Userguide.pdf pp 33
```
hmmpress ribosomal_hmm
```

### Blastrules

Download proteins and blastrules:
    ```
    https://ftp.ncbi.nih.gov/pub/blastrules/RELEASE_4.0/
    ```

In [29]:
blastrules = pd.read_csv('/data/mhoffert/db/pgap_ribosomal/blastrules_4.0/blast-rules_4.0.tsv', sep='\t', encoding='latin1')

In [30]:
ribosomal_blastrules = blastrules[blastrules['Name'].apply(str.lower).str.contains('ribosomal protein')]

In [31]:
# count = 0
# potential_blastrules = blastrules[blastrules['Name'].apply(str.lower).str.contains('ribosomal protein')]
# validated_blastrules = []
# for index, row in potential_blastrules.iterrows():
    
#     display(f'{count} out of {len(potential_blastrules)}')
#     display(row['Name'])
#     display(row['Gene'])
#     is_ribosomal = ''
#     while not is_ribosomal in ['y', 'n']:
#         is_ribosomal = input("Is this a ribosomal protein? y/n:")
#         display('Try again')
    
#     if is_ribosomal == 'y':
#         validated_blastrules.append(row)
#     clear_output(wait=True)
#     count += 1

# ribosomal_blastrules = pd.concat(validated_blastrules, axis=1).T

# ribosomal_blastrules.to_csv('/data/mhoffert/realmichaelhoffert/ribosomal_proteins/data/validated_ribosomal_proteins_blastrules.tsv', sep='\t')

In [32]:
ribosomal_blastrules_accs = [p for plist in ribosomal_blastrules['WP_Proteins'].apply(lambda x: x.split(',')) for p in plist]

In [33]:
with open('/data/mhoffert/db/pgap_ribosomal/blastrules_4.0/proteins.fasta', 'r') as proteins_fasta:
    records = [r for r in SeqIO.parse(proteins_fasta, format='fasta')]

In [34]:
ribosomal_records = [r for r in records if any(s in r.id for s in ribosomal_blastrules_accs)]

In [36]:
with open('/data/mhoffert/db/pgap_ribosomal/blastrules_4.0/ribosomal_proteins.fasta', 'w') as ribosomal_proteins_fasta:
    SeqIO.write(ribosomal_records, ribosomal_proteins_fasta, 'fasta')

```
makeblastdb -in ribosomal_proteins.fasta -dbtype prot -out ribosomal_proteins
```